## Ideas for visualisation :
* maps of users
    * scaled with nb followers 
    * scaled with nb stars
* commit / user heatmap
* dominant languages per canton
* Cooperation graph (find which unis have common projects)
* map of repos

In [1]:
%matplotlib inline

import pandas as pd
import itertools

from utils import get_mongo_db

In [2]:
db = get_mongo_db()

Connecting to MongoDB at localhost:27017...


In [3]:
repos = db.repositories.find({ 'language': { '$ne': None } }, { 'language': True })
repos.count()

184059

In [4]:
res = db.users.find({ 'in_ch': True, 'repositories': { '$ne': None } })

users = []

for user in res:
    repos = db.repositories.find(
        { '_id': { '$in': user['repositories'] } }
    )
    
    users.append({
        '_id': user['_id'],
        'login': user['login'],
        'name': user['name'],
        'location': user['location'],
        'repositories_docs': list(repos)
    })

In [6]:
users[0]

{'_id': 2022803,
 'location': 'Switzerland',
 'login': 'vhf',
 'name': 'victor felder',
 'repositories_docs': [{'_id': 5143315,
   'clone_url': 'https://github.com/vhf/pareidoloop.git',
   'created_at': datetime.datetime(2012, 7, 22, 17, 33, 58),
   'fork': True,
   'forks_count': 4,
   'full_name': 'vhf/pareidoloop',
   'has_downloads': True,
   'has_issues': False,
   'has_pages': False,
   'homepage': '',
   'language': 'JavaScript',
   'name': 'pareidoloop',
   'open_issues': 0,
   'open_issues_count': 0,
   'owner_id': 2022803,
   'private': False,
   'size': 150,
   'stargazers_count': 11,
   'watchers_count': 11},
  {'_id': 5493188,
   'clone_url': 'https://github.com/vhf/vhf.github.io.git',
   'created_at': datetime.datetime(2012, 8, 21, 9, 52, 28),
   'fork': False,
   'forks_count': 3,
   'full_name': 'vhf/vhf.github.io',
   'has_downloads': True,
   'has_issues': True,
   'has_pages': True,
   'homepage': '',
   'language': 'HTML',
   'name': 'vhf.github.io',
   'open_issues

In [8]:
def count_repos(user):
    return len(user['repositories_docs'])

def count_stat(user, key):
    count = 0
    for repo in user['repositories_docs']:
        count = count + repo[key]
    return count

def count_stars(user):
    return count_stat(user, 'stargazers_count')

def count_watchers(user):
    return count_stat(user, 'watchers_count')

def count_forks(user):
    return count_stat(user, 'forks_count')



In [10]:
users_data = [{ 'id': user['_id'], \
                'location': user['location'], \
                'name' : user['name'], \
                'repo_count' : count_repos(user), \
                'star_count' : count_stars(user)\
              } \
              for user in users]
users_df = pd.DataFrame(users_data)